In [1]:
!pip install tensorflow numpy matplotlib opencv-python scikit-learn


In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split


In [2]:
# Define the directory path where your data is stored
data_dir = '/Users/ihortresnystkyi/train_v2'

# Load the CSV file
df = pd.read_csv(os.path.join(data_dir, 'train_ship_segmentations_v2.csv'))

# Initialize lists to store the images and masks
images = []
masks = []

In [3]:
# Define a function to convert RLE to mask
def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction


In [ ]:
# Loop over the rows of the DataFrame
for idx, row in df.iterrows():
    # Get the image ID and the RLE mask
    image_id = row['ImageId']
    rle_mask = row['EncodedPixels']

    if isinstance(rle_mask, str): # Only process images that have ship
        # Decode the RLE mask
        mask = rle_decode(rle_mask)

        # Open the image file
        image = Image.open(os.path.join(data_dir, f'{image_id}'))

        # Append the image and mask to the lists
        images.append(np.array(image))
        masks.append(mask)

    # Print a progress message every 1000 images
    if idx % 1000 == 0:
        print(f"Processed {idx} images")

# Convert lists to numpy arrays
print("Converting lists to arrays")
X = np.array(images)
Y = np.array(masks)[..., np.newaxis]

Processed 0 images
Processed 1000 images
Processed 2000 images
Processed 3000 images
Processed 4000 images
Processed 5000 images
Processed 6000 images
Processed 7000 images
Processed 8000 images
Processed 9000 images
Processed 10000 images
Processed 11000 images
Processed 12000 images
Processed 13000 images
Processed 14000 images
Processed 15000 images
Processed 16000 images
Processed 17000 images
Processed 18000 images
Processed 19000 images
Processed 20000 images
Processed 21000 images
Processed 22000 images
Processed 23000 images
Processed 24000 images
Processed 25000 images
Processed 26000 images
Processed 27000 images
Processed 28000 images
Processed 29000 images
Processed 30000 images
Processed 31000 images
Processed 32000 images
Processed 33000 images
Processed 34000 images
Processed 35000 images
Processed 36000 images
Processed 37000 images
Processed 38000 images
Processed 39000 images
Processed 40000 images
Processed 41000 images
Processed 42000 images
Processed 43000 images
P

In [3]:
print("Splitting data into train and validation sets")
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


Splitting data into train and validation sets


NameError: name 'X' is not defined

In [ ]:
def unet_model(input_size=(768, 768, 1)):
    inputs = Input(input_size)
    
    # Downsampling path
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Upsampling path
    up1 = UpSampling2D(size=(2, 2))(pool1)
    merge1 = concatenate([conv1, up1], axis=3)
    conv2 = Conv2D(1, (1, 1), activation='sigmoid')(merge1)
    
    return Model(inputs=inputs, outputs=conv2)


In [ ]:
# Define a custom metric (Dice coefficient)
def dice_coef(y_true, y_pred, smooth=1):
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    return tf.reduce_mean((2. * intersection + smooth) / (union + smooth))

# Create and compile the model
print("Creating and compiling the model")
model = unet_model()
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=[dice_coef])

# Define the model checkpoint
checkpoint = ModelCheckpoint('unet.hdf5', monitor='val_loss', save_best_only=True)

# Train the model
print("Training the model")
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=50, callbacks=[checkpoint])


In [ ]:
# Compile the model
model = unet_model()
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[dice_coef])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Save the model
model.save('model.h5')

In [ ]:
# Assuming test_images is your array of test images and pred_masks is your array of predicted masks
for i in range(len(test_images)):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    # Show original image
    ax[0].imshow(test_images[i], cmap='gray')
    ax[0].set_title('Original Image')

    # Show mask (predicted by the model)
    ax[1].imshow(np.squeeze(pred_masks[i]), cmap='gray')
    ax[1].set_title('Predicted Mask')

    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from keras.models import load_model
from base_ship.train_model import dice_coef

# Load the trained model
model = load_model('model.h5', custom_objects={'dice_coef': dice_coef})

# Define the directory path where your data is stored
data_dir = 'actual_path'

# Load the CSV file
df_test = pd.read_csv(os.path.join(data_dir, 'test_ship_segmentations_v2.csv'))

# Initialize a list to store the images
images_test = []

# Loop over the rows of the DataFrame
for idx, row in df_test.iterrows():
    # Get the image ID
    image_id = row['ImageId']

    # Open the image file
    image = Image.open(os.path.join(data_dir, f'{image_id}'))

    # Append the image to the list
    images_test.append(np.array(image))

# Convert the list to a numpy array
X_test = np.array(images_test)

# Use the model to make predictions
predictions = model.predict(X_test)